In [1]:
# Load libs:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from glob import glob
from os.path import dirname, abspath, join, exists

from clearml import Task

from sklearn import linear_model

from amplify.data import DataGenerator, DataSplit

In [2]:
# ClearML Stuff
task = Task.init(project_name="amplify", task_name="predict_power_gen_initial_model")

ClearML Task: overwriting (reusing) task id=4516e4ddc8d14c499b6ba144100f5b7a
2022-01-29 17:58:23,471 - clearml.Task - WARNING - Failed auto-detecting task repository: Script file ['ipykernel_launcher', '/var/folders/14/343hdqps63j446zd_0kxgvwm0000gn/T/ipykernel_11917/3907652897.py'] could not be found
ClearML results page: https://app.clear.ml/projects/f0e0b096252d4c18b9005e580afc5bce/experiments/4516e4ddc8d14c499b6ba144100f5b7a/output/log


## Load Data

In [3]:
# DataGenerator - ClearML
xy_data = DataGenerator().load_data()
xy_data.head()

ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
Info: Successfully loaded building data!
Info: Successfully loaded weather data!


,temp,pressure,humidity,clouds_all,irradiance,day_of_week,solar,usage
2021-10-19 04:00:00+00:00,10.07,1017.0,66.0,20.0,0.0,2,1.0,11.3
2021-10-19 05:00:00+00:00,9.51,1017.0,70.0,20.0,0.0,2,1.0,13.2
2021-10-19 06:00:00+00:00,9.28,1017.0,71.0,20.0,0.0,2,1.0,12.7
2021-10-19 07:00:00+00:00,9.12,1017.0,72.0,20.0,0.0,2,1.0,17.2
2021-10-19 08:00:00+00:00,9.12,1017.0,73.0,20.0,0.0,2,1.0,17.0


In [4]:
# What is the shape of the data?
print(f"XY Data Shape: {xy_data.shape}")

XY Data Shape: (1994, 8)


In [5]:
# Lets see a few more rows
xy_data.head(20)

,temp,pressure,humidity,clouds_all,irradiance,day_of_week,solar,usage
2021-10-19 04:00:00+00:00,10.07,1017.0,66.0,20.0,0.000000,2,1.0,11.3
2021-10-19 05:00:00+00:00,9.51,1017.0,70.0,20.0,0.000000,2,1.0,13.2
2021-10-19 06:00:00+00:00,9.28,1017.0,71.0,20.0,0.000000,2,1.0,12.7
2021-10-19 07:00:00+00:00,9.12,1017.0,72.0,20.0,0.000000,2,1.0,17.2
2021-10-19 08:00:00+00:00,9.12,1017.0,73.0,20.0,0.000000,2,1.0,17.0
2021-10-19 09:00:00+00:00,8.91,1017.0,75.0,1.0,0.000000,2,1.0,14.2
2021-10-19 10:00:00+00:00,8.61,1018.0,75.0,1.0,0.000000,2,1.0,14.4
2021-10-19 11:00:00+00:00,8.97,1018.0,73.0,1.0,0.000000,2,1.0,14.1
2021-10-19 12:00:00+00:00,9.12,1018.0,75.0,1.0,323.335022,2,11.3,12.5
2021-10-19 13:00:00+00:00,10.96,1019.0,69.0,1.0,680.919505,2,24.9,13.7


In [6]:
# timestamp example
xy_data.index[0]

Timestamp('2021-10-19 04:00:00+0000', tz='UTC', freq='H')

In [7]:
# Data Shape 
# Batch Size of 10
# Series of 7 Days 168
# (B,  S,   H, F) <-X Y-> (B,  S,   H, O)
# (20, 30, 48, 5) <-X Y-> (20, 30, 48, 1)

## Data Processing

```
1. Make xy data into a set of series of shape [N, M, F],
where N = number of examples; M = nubmer of timestamps; F = Number of features
```

```
2. Train-Test-Validation Split. Split off at 0.8, 0.1, 0.1 ratio for now. Shuffle the split.
```

```
3. Seperate x, y from dataset. Remove last column from dataset to extract y.
```

In [8]:
# Data Shape 
# Batch Size of 10
# Series of 7 Days 168
# (B,  S,   H, F) <-X Y-> (B,  S,   H, O)
# (20, 30, 48, 5) <-X Y-> (20, 30, 48, 1)
train_ds, val_ds, test_ds = DataSplit(xy_data).split_data()

In [9]:
# Isolate XY from datasets
x_train, y_solar_train, y_usage_train = (
    train_ds[0].astype("float32"),
    train_ds[1].astype("float32"),
    train_ds[2].astype("float32"),
)  # Training data
x_val, y_solar_val, y_usage_val = (
    val_ds[0].astype("float32"),
    val_ds[1].astype("float32"),
    val_ds[2].astype("float32"),
)  # Validation data
x_test, y_solar_test, y_usage_test = (
    test_ds[0].astype("float32"),
    test_ds[1].astype("float32"),
    test_ds[2].astype("float32"),
)  # Test data


In [10]:
# format data for training...
# [1,2,....] <= was a column
# do we do (n, 1, n timesteps, n features per timestep) (3,4,5 => 2 days)?

### Reviewing Data

In [11]:
# x_train shape
print("x_train shape")
x_train.shape

x_train shape


(519, 48, 6)

In [12]:
# y_train shape
print("y_solar_train shape")
y_solar_train.shape

y_solar_train shape


(519, 48, 1)

## Modeling

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from keras import Input
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.layers import Bidirectional
from keras.layers import TimeDistributed

### Normalize Data

In [14]:
#TODO: Move to model pipeline
# Normalize X_train data
norm_layer = tf.keras.layers.Normalization(axis=-1)
norm_layer.adapt(x_train)

2022-01-29 17:59:06.702689: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Simple LSTM

In [15]:
def CreateModel(n_layer=1, n_series_len=48, n_ft=6, n_outputs=1):
    lstm_input = Input(shape=(n_series_len, n_ft))
    lstm_layer = LSTM(n_layer, activation='relu')(lstm_input)
    x = Dense(n_outputs)(lstm_layer)
    model = Model(inputs=lstm_input, outputs=x)
    return model

In [16]:
n_layer = 6
n_series_len = 48
n_ft = 6
n_outputs = 1
model = CreateModel(n_layer, n_series_len, n_ft, n_outputs)

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 6)]           0         
                                                                 
 lstm (LSTM)                 (None, 6)                 312       
                                                                 
 dense (Dense)               (None, 1)                 7         
                                                                 
Total params: 319
Trainable params: 319
Non-trainable params: 0
_________________________________________________________________


In [18]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
#model.compile(loss=keras.losses.MeanAbsoluteError(), optimizer=optimizer)
model.compile(tf.optimizers.Adam(learning_rate=0.0003),
      loss='binary_crossentropy',
      metrics=['acc'])

In [19]:
history = model.fit(
    x=x_train,
    y=y_solar_train,
    epochs=100,
    batch_size=10,
    validation_data=(x_val, y_solar_val),
    shuffle=False,
)


Epoch 1/100
52/52 [==============================] - 2s 14ms/step - loss: 106.6964 - acc: 0.0000e+00 - val_loss: 96.3644 - val_acc: 0.0000e+00
Epoch 2/100
52/52 [==============================] - 0s 9ms/step - loss: 106.6964 - acc: 0.0000e+00 - val_loss: 96.3644 - val_acc: 0.0000e+00
Epoch 3/100
52/52 [==============================] - 0s 9ms/step - loss: 106.6964 - acc: 0.0000e+00 - val_loss: 96.3644 - val_acc: 0.0000e+00
Epoch 4/100
52/52 [==============================] - 0s 9ms/step - loss: 106.6964 - acc: 0.0000e+00 - val_loss: 96.3644 - val_acc: 0.0000e+00
Epoch 5/100
52/52 [==============================] - 0s 9ms/step - loss: 106.6964 - acc: 0.0000e+00 - val_loss: 96.3644 - val_acc: 0.0000e+00
Epoch 6/100
52/52 [==============================] - 0s 9ms/step - loss: 106.6964 - acc: 0.0000e+00 - val_loss: 96.3644 - val_acc: 0.0000e+00
Epoch 7/100
52/52 [==============================] - 0s 9ms/step - loss: 106.6964 - acc: 0.0000e+00 - val_loss: 96.3644 - val_acc: 0.0000e+00
Epoch